# Create Pseudo FGS image out of an `ID_image_cal.fits` image

This notebook will be used during the case that we want to check the 3x3 count rates from the pseudo-FGS image created from the NIRCam image against FGS ID strips.

This will be important if the last guiding attempt failed due to a mismatch between the commanded count rates and the measured countrates. At that time we will retrieve the ID merged image from MAST from the last guiding attempt and use it to determine what the 3x3 count rates are for the commanded PSFs. The pseudo-FGS image should not be used to create a new command, instead, the measured 3x3 count rates can be used in place of those in the previous command; be sure to update the threshold as well. 

In [ ]:
import os
from functools import reduce
import logging
import shutil

from astropy.io import fits
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import numpy as np
import pandas as pd
from IPython.display import Image

import fgscountrate
import jwst_magic
from jwst_magic.convert_image import convert_image_to_raw_fgs
from jwst_magic.utils import utils
PACKAGE_PATH = jwst_magic.__file__.split('jwst_magic/__init__.py')[0]

LOGGER = logging.getLogger('convert_merged_id_strips_to_raw-notebook')

%matplotlib inline

# Table of Contents
1. [About this Observation](#About-this-Observation)
2. [Segment ID matching to MAGIC labeling](#Segment-ID-matching-to-MAGIC-labeling)
2. [Functions](#Functions)
3. [Load in the ID strips merged image](#Load-in-the-ID-strips-merged-image)
4. [Run this image through the backend of MAGIC to create a pseudo-FGS image](#Run-this-image-through-the-backend-of-MAGIC-to-create-a-pseudo-FGS-image)

# About this Observation

Edit the cell below with the guider used for this observations

In [ ]:
guider = 1

# Segment ID matching to MAGIC labeling
Run but do not change these cells

In [ ]:
if guider==1:
    orientation_filename = "../Commissioning_Support/fgs_raw_orientation_numbering_wss_guider1.png"  
elif guider==2:
    orientation_filename="../Commissioning_Support/fgs_raw_orientation_numbering_wss_guider2.png"
    
magic_grid_filename = "../JWSTgrid_side.png"

In [ ]:
print(f'Guider {guider}')

In [ ]:
Image(filename=orientation_filename, height=400, width=400)

In [ ]:
Image(filename=magic_grid_filename, height=400, width=400)

In [ ]:
# MAGIC segment labels to segment naming G1 map
SEGMENT_MAP_G1 = {"A":"B4-13", "B":"C4-14", "C":"C3-12", "D":"B5-15",
                  "E":"A4-4", "F":"B3-11", "G":"A5-5", "H":"A3-3", 
                  "I":"C5-16", "J":"C2-10", "K":"A6-6", "L":"A2-2", 
                  "M":"B6-17", "N":"A1-1", "O":"B2-9", "P":"C6-18", 
                  "Q":"C1-8", "R":"B1-7"}

# MAGIC segment lbabels to segment naming G2 map
SEGMENT_MAP_G2 = {"A":"B1-7", "B":"C6-18", "C":"C1-8", "D":"B6-17",
                  "E":"A1-1", "F":"B2-9", "G":"A6-6", "H":"A2-2", 
                  "I":"C5-16", "J":"C2-10", "K":"A5-5", "L":"A3-3", 
                  "M":"B5-15", "N":"A4-4", "O":"B3-11", "P":"C4-14", 
                  "Q":"C3-12", "R":"B4-13"}

# Functions 
Do not change these cells

In [ ]:
def read_all_found_psfs(all_psfs_filename):
    """
    Read in the all found PSFs file from MAGIC and map the segment ID to the measured PSF. Return a data frame 
    (table) with the information in this file
    """
    # Read in all found PSFs
    segment_map = SEGMENT_MAP_G1 if guider==1 else SEGMENT_MAP_G2
    
    all_psfs_table = pd.read_csv(all_psfs_filename, comment='#', 
                                 names=['label', 'y', 'x', 'countrate'], sep=' ')
    segments = [segment_map[label] for label in all_psfs_table['label']]

    # Add the segment name to the table
    all_psfs_table.insert(1, 'segment', segments)
    all_psfs_table = all_psfs_table.sort_values('segment', ascending=True)
    
    return all_psfs_table

In [ ]:
def convert_id_to_fgs_raw(input_image, root, out_dir, guider, smoothing):
    # Determine filename root
    root = utils.make_root(root, input_image)

    # Determine output directory
    out_dir_root = utils.make_out_dir(out_dir, PACKAGE_PATH, root)
    utils.ensure_dir_exists(out_dir_root)

    # Set up logging
    _, log_filename = utils.create_logger_from_yaml(__name__, out_dir_root=out_dir_root, root=root, level='DEBUG')

    LOGGER.info("Package directory: {}".format(PACKAGE_PATH))
    LOGGER.info("Processing request for {}.".format(root))
    LOGGER.info("All data will be saved in: {}".format(out_dir_root))
    LOGGER.info("Input image: {}".format(os.path.abspath(input_image)))

    # Copy input image into out directory
    try:
        shutil.copy(os.path.abspath(input_image), out_dir_root)
    except shutil.SameFileError:
        pass

    # Convert provided FGS ID image to a "raw" FGS image
    fgs_im, all_found_psfs_file, psf_center_file, fgs_hdr_dict = \
        convert_image_to_raw_fgs.convert_im(input_image, guider, root,
                                            out_dir=out_dir,
                                            nircam=False,
                                            nircam_det=None,
                                            normalize=False,
                                            smoothing=smoothing,
                                            coarse_pointing=False,
                                            jitter_rate_arcsec=None,
                                            logger_passed=True,
                                            itm=False)

    # Add logging information to fgs image header
    fgs_hdr_dict['LOG_FILE'] = (os.path.basename(log_filename), 'Log filename')

    # Write converted image
    convert_image_to_raw_fgs.write_fgs_im(fgs_im, out_dir, root, guider, fgs_hdr_dict)
    LOGGER.info("*** Image Conversion COMPLETE ***")


# Load in the ID strips merged image

We will be loading the `gs-id_<guider>_image_cal.fits` associated with the observation we care about. This a merged ID strips image. It has the shape 2048x2024 so you will need to pad it by 24 rows of zeros. Since the positions of the PSFs don't matter in the image, this change will not negatively impact the data or our analysis. 

In [ ]:
##############################################################################
### UPDATE the path below to match where the image_cal.fits image is saved ###
##############################################################################

# Grab the *_gs-id_1_image_cal.fits image associated with your CAR/obs from MAST
# and provide the path to this image below

path_to_id_image_cal = '/Users/kbrooks/Documents/tel/FGS/sample_images/jw00646183001_02101_00002_nrcblong/jw00646183001_gs-id_1_image_cal.fits'

In [ ]:
hdu = fits.open(path_to_id_image_cal)
hdu.info()

In [ ]:
sci = fits.getdata(path_to_id_image_cal)
print(np.shape(sci))

plt.figure(figsize=(10, 8))
plt.imshow(sci[0], norm=LogNorm())
plt.colorbar()
plt.show()

In [ ]:
# Subtract off the median of the image to remove as much background as possible
bk_sub_sci = sci[0] - np.median(np.asarray(sci[0])) 

#####################################################
### UPDATE the variables below to match your data ###
#####################################################
center_of_config = (690, 1000) # Eye ball the center of the guide star PSF configuration
radius_of_window = 500 # The half size of the window around the guide star PSF configuration

# Crop out the guide star PSF configuration
cropped = bk_sub_sci[center_of_config[1]-radius_of_window: center_of_config[1]+radius_of_window,
                     center_of_config[0]-radius_of_window: center_of_config[0]+radius_of_window]

In [ ]:
plt.figure(figsize=(10,8))
plt.imshow(cropped, norm=LogNorm())
plt.colorbar()
plt.show()

In [ ]:
# Pad the image to a 2048x2048 image
pad = (2048 - (2*radius_of_window)) // 2
padded_sci = np.pad(cropped, pad, 'constant')

if np.shape(padded_sci) != (2048, 2048):
    print(f'The shape of the input data is not as expected ({np.shape(padded_sci)}, adjust the padding function accordingly')

In [ ]:
# Make sure that the associated data is the same shape as the SCI image
err = hdu['ERR'].data
padded_err = np.pad(err[0], ((0,0), (12,12)), 'constant')
if np.shape(padded_err) != (2048, 2048):
    print('The shape of the input data is not as expected, adjust the padding function accordingly')

dq = hdu['DQ'].data
padded_dq = np.pad(dq, ((0,0), (12,12)), 'constant')
if np.shape(padded_dq) != (2048, 2048):
    print('The shape of the input data is not as expected, adjust the padding function accordingly')

In [ ]:
# Put these back into the file that we will be using and write it out
hdu['SCI'].data = padded_sci
hdu['ERR'].data = padded_err
hdu['DQ'].data = padded_dq

path, filename = os.path.split(path_to_id_image_cal)
new_filename = f'expanded_{filename}'
outfile = os.path.join('/Users/kbrooks/Desktop', new_filename)
hdu.writeto(outfile, overwrite=True)

# Run this image through the backend of MAGIC to create a pseudo-FGS image 

In [ ]:
# Be sure to change the root and out_dir as it relates to your observation
# Input image and paths
input_image = outfile # This is the same image that you just wrote out
root = 'from_ote07_obs1_id' # Change this root to something that makes sense for this observation
out_dir = '/Users/kbrooks/Desktop/MAGIC/test/' # Change this path

# Smoothing information
smoothing = 'high' #"high" for large smoothing (e.g. Global Alignment), "default" for medium smoothing 

In [ ]:
fits.info(input_image)

In [ ]:
# Create a peusdo-FGS image from the ID image
convert_id_to_fgs_raw(input_image, root, out_dir, guider, smoothing)

In [ ]:
# Make a table of the all found PSFs
all_psfs_filename = os.path.join(out_dir, 'out', root, f'unshifted_all_found_psfs_{root}_G{guider}.txt')
all_psfs_table = read_all_found_psfs(all_psfs_filename)

all_psfs_table

### Do a quick visual check to make sure that the identified segments are correct 

In [ ]:
pseudo_fgs = fits.getdata(os.path.join(out_dir, 'out', root, 'FGS_imgs', 
                                       f'unshifted_{root}_G{guider}.fits'))

In [ ]:
xs = all_psfs_table['x'].values
ys = all_psfs_table['y'].values
labels = all_psfs_table['label'].values
segments = all_psfs_table['segment'].values

plt.figure(figsize=(10, 8))
plt.imshow(pseudo_fgs, cmap='Blues_r', vmin=0, vmax=1000, origin='upper')
plt.title(f"Pseudo FGS image from FGS")
for segment, label, y, x in zip(segments, labels, ys, xs):
    plt.annotate(label, (x, y), (x+5, y+5), color='white', fontsize=12)
    plt.annotate(segment, (x, y), (x-40, y-60), color='cyan', fontsize=12)
plt.show()

In [ ]:
Image(filename=orientation_filename, height=400, width=400) 

Confirm that the segments found are consistent with the diagrams above (depending on which guider is being used) and matches what is given in the all_psfs_table for the pseudo-FGS image from the ID strips. If you need to update the segment names in the table, you can do so by creating a list of the actual segment IDs in order as they appear in the table (for example called `actual_segments` and run a cell with the following lines of code: 
```
actual_segments = [,] # Fill in this list
all_psfs_table['segment'] = actual_segments
```

In [ ]:
all_psfs_table

# About this notebook

Author: K. Brooks

Last updated: 13 Jan, 2022